In [49]:
import numpy as np
import pandas as pd 
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [50]:
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_path = "data/Auto_MPG_data.csv"
dataset = pd.read_csv(dataset_path)

In [52]:
X = dataset.drop(columns="MPG").values
y = dataset["MPG"].values

In [53]:
val_size = 0.2
test_size = 0.125
shuffle = True 

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=val_size,
    random_state=random_state,
    shuffle=shuffle
)

X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val,
    test_size=test_size,
    random_state=random_state,
    shuffle=shuffle
)

In [54]:
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [55]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X 
        self.y = y 
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx] 

In [56]:
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [57]:
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
        
    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        out = self.output(x)
        return out.squeeze(1)

In [58]:
input_dims = X_train.shape[1]
output_dims = 1
hidden_dims = 64

model = MLP(input_dims, hidden_dims, output_dims).to(device)

In [59]:
lr = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [60]:
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true)**2)
    ss_res = torch.sum((y_true - y_pred)**2)
    r2 = 1 - ss_res / ss_tot
    return r2 

In [61]:
epochs = 100
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []

for epoch in range(epochs):
    train_loss = 0.0
    train_target = []
    val_target = []
    train_predict = []
    val_predict = []
    model.train()
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        optimizer.zero_grad()
        outputs = model(X_samples)
        
        train_predict += outputs.tolist()
        train_target += y_samples.tolist()
        
        loss = criterion(outputs, y_samples)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss) 
    train_r2.append(r_squared(train_target, train_predict))
    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            outputs = model(X_samples)
            
            val_predict += outputs.tolist()
            val_target += y_samples.tolist()
            
            loss = criterion(outputs, y_samples)
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_r2.append(r_squared(val_target, val_predict))
    
    print(f"Epoch: {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch: 1/100, Train Loss: 370.1839, Val Loss: 366.8093
Epoch: 2/100, Train Loss: 108.0241, Val Loss: 23.3887
Epoch: 3/100, Train Loss: 117.7639, Val Loss: 8.5808
Epoch: 4/100, Train Loss: 15.8566, Val Loss: 90.7862
Epoch: 5/100, Train Loss: 21.2842, Val Loss: 43.9200
Epoch: 6/100, Train Loss: 26.3672, Val Loss: 27.5693
Epoch: 7/100, Train Loss: 22.0639, Val Loss: 77.7907
Epoch: 8/100, Train Loss: 19.0946, Val Loss: 19.4210
Epoch: 9/100, Train Loss: 18.0524, Val Loss: 4.5430
Epoch: 10/100, Train Loss: 13.5312, Val Loss: 4.8618
Epoch: 11/100, Train Loss: 14.0606, Val Loss: 4.9649
Epoch: 12/100, Train Loss: 9.7093, Val Loss: 17.7016
Epoch: 13/100, Train Loss: 15.0320, Val Loss: 9.3601
Epoch: 14/100, Train Loss: 16.5545, Val Loss: 4.1752
Epoch: 15/100, Train Loss: 7.9053, Val Loss: 4.9917
Epoch: 16/100, Train Loss: 12.1479, Val Loss: 3.7954
Epoch: 17/100, Train Loss: 8.6331, Val Loss: 6.0187
Epoch: 18/100, Train Loss: 10.6782, Val Loss: 17.4772
Epoch: 19/100, Train Loss: 11.9595, Val Loss:

In [62]:
test_target = []
test_predict = []

model.eval()

with torch.no_grad():
    for X_samples, y_samples in test_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        outputs = model(X_samples)
        
        test_predict += outputs.tolist()
        test_target += y_samples.tolist()
        
test_r2 = r_squared(test_target, test_predict)

print(f"Test R2: {test_r2}")

Test R2: 0.7950974702835083
